In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.metrics import classification_report, accuracy_score, mean_squared_error, mean_absolute_error
from sklearn.utils.class_weight import compute_class_weight
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
# from tensorflow.keras.losses import MeanSquaredError, MeanAbsoluteError
from tensorflow.keras.layers import Dense, LSTM, Input, Concatenate, BatchNormalization
from tensorflow.keras.layers import Dropout, LeakyReLU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import EarlyStopping
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import RandomizedSearchCV
from google.colab import drive
import json
import shutil

# Mount Google Drive
drive.mount('/content/drive')

# Path folder di Google Drive untuk menyimpan model
drive_path = "/content/drive/My Drive/Colab Notebooks/Ekspor Models"

# Pastikan folder tujuan ada di Google Drive
import os
if not os.path.exists(drive_path):
    os.makedirs(drive_path)

!pip install --upgrade tensorflow

!python --version

print(f"Versi TensorFlow: {tf.__version__}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Python 3.10.12
Versi TensorFlow: 2.18.0



### 1. **Import Library**
```python
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.metrics import classification_report, accuracy_score, mean_squared_error, mean_absolute_error
from sklearn.utils.class_weight import compute_class_weight
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, LSTM, Input, Concatenate, BatchNormalization
from tensorflow.keras.layers import Dropout, LeakyReLU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import EarlyStopping
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import RandomizedSearchCV
from google.colab import drive
import json
import shutil
```

- **`pandas`**: Digunakan untuk membaca dan memproses data dalam format tabular seperti `.csv`.
- **`numpy`**: Digunakan untuk operasi matematika dan array numerik.
- **`sklearn.model_selection`**:
  - `train_test_split`: Untuk membagi dataset menjadi data latih dan validasi.
- **`sklearn.preprocessing`**:
  - `MinMaxScaler`: Normalisasi fitur ke rentang tertentu (0 hingga 1).
  - `LabelEncoder`: Mengubah label kategori menjadi angka.
- **`sklearn.metrics`**: Untuk menghitung metrik evaluasi model seperti akurasi dan error.
- **`compute_class_weight`**: Menghitung bobot kelas untuk menangani ketidakseimbangan data.
- **`tensorflow`**:
  - Framework untuk membangun model deep learning.
  - **`keras`** digunakan untuk membuat layer seperti `LSTM`, `Dense`, dan optimasi.
- **`seaborn`**: Untuk membuat visualisasi data.
- **`matplotlib.pyplot`**: Untuk membuat grafik dan plot.
- **`os`**: Mengelola sistem file seperti membuat folder.
- **`shutil`**: Memindahkan atau menyalin file.
- **`json`**: Membaca dan menulis data dalam format JSON.
- **`google.colab.drive`**: Menghubungkan Google Colab dengan Google Drive untuk menyimpan model.

---

### 2. **Mount Google Drive**
```python
drive.mount('/content/drive')
```
- Menghubungkan Google Colab dengan akun Google Drive pengguna. File dapat disimpan atau diakses dari Drive setelah ini.

---

### 3. **Path Folder di Google Drive**
```python
drive_path = "/content/drive/My Drive/Colab Notebooks/Ekspor Models"
```
- Menentukan lokasi penyimpanan file model atau hasil lainnya di Google Drive.

---

### 4. **Pastikan Folder Tujuan Ada**
```python
import os
if not os.path.exists(drive_path):
    os.makedirs(drive_path)
```
- Mengecek apakah folder tujuan (`drive_path`) sudah ada. Jika belum, membuat folder baru di lokasi tersebut.

---

### 5. **Install TensorFlow**
```python
!pip install --upgrade tensorflow
```
- Memastikan TensorFlow di Colab adalah versi terbaru dengan cara meng-upgrade library TensorFlow.

---

### 6. **Tampilkan Versi Python**
```python
!python --version
```
- Menampilkan versi Python yang sedang digunakan oleh Colab.

---

### 7. **Cek Versi TensorFlow**
```python
print(f"Versi TensorFlow: {tf.__version__}")
```
- Menampilkan versi TensorFlow yang saat ini diinstall. Penting untuk kompatibilitas kode dengan TensorFlow.

---

### **Penjelasan Kode Tambahan**
- **Menggunakan TensorFlow untuk Deep Learning**:
  - Layer seperti `LSTM` digunakan untuk analisis data sekuensial.
  - Layer `Dense` digunakan untuk membuat lapisan fully connected pada neural network.
  - Optimizers seperti `Adam` membantu meminimalkan fungsi loss selama pelatihan.

- **Mengelola Data dengan `pandas`**:
  - `pandas` digunakan untuk membaca dataset (`.csv`) dan memproses data untuk diinputkan ke dalam model.

- **Evaluasi Model**:
  - Menggunakan metrik seperti akurasi (`accuracy_score`) dan laporan klasifikasi (`classification_report`) untuk mengukur performa model.

---


In [ ]:
# Path dataset
dataset_path_data_gizi_anak = "/content/drive/My Drive/Colab Notebooks/Datasets/gizi_anak_indonesia_balanced_1000.csv"
dataset_path_food_data = "/content/drive/My Drive/Colab Notebooks/Datasets/FOOD-DATA-GROUP5-CLEANED.csv"
# z_score_data_path = "/content/drive/My Drive/Colab Notebooks/Datasets/z_score_data_who.csv"

# Membaca dataset
growth_data = pd.read_csv(dataset_path_data_gizi_anak)
food_data = pd.read_csv(dataset_path_food_data)
# z_score_data = pd.read_csv(z_score_data_path)


---

### **1. Path Dataset**
```python
dataset_path_data_gizi_anak = "/content/drive/My Drive/Colab Notebooks/Datasets/gizi_anak_indonesia_balanced_1000.csv"
dataset_path_food_data = "/content/drive/My Drive/Colab Notebooks/Datasets/FOOD-DATA-GROUP5-CLEANED.csv"
# z_score_data_path = "/content/drive/My Drive/Colab Notebooks/Datasets/z_score_data_who.csv"
```

- **`dataset_path_data_gizi_anak`**:
  - Menyimpan path file dataset yang berisi data status gizi anak.
  - File `gizi_anak_indonesia_balanced_1000.csv` berisi informasi seperti usia, tinggi badan, berat badan, serta status gizi anak dalam format CSV (Comma-Separated Values).

- **`dataset_path_food_data`**:
  - Menyimpan path file dataset yang berisi data terkait makanan dan kandungan nutrisinya.
  - File `FOOD-DATA-GROUP5-CLEANED.csv` dapat digunakan untuk menganalisis rekomendasi makanan berdasarkan status gizi anak.

- **`z_score_data_path` (di-comment)**:
  - Path untuk dataset referensi WHO yang berisi data z-score untuk menganalisis pertumbuhan anak.
  - Data ini membantu menghitung status gizi anak dengan membandingkan BMI anak terhadap data referensi yang sesuai.

**Catatan**:
- Pastikan file pada path yang disebutkan telah diunggah ke Google Drive dengan struktur yang sama agar dapat diakses.

---

### **2. Membaca Dataset**
```python
growth_data = pd.read_csv(dataset_path_data_gizi_anak)
food_data = pd.read_csv(dataset_path_food_data)
z_score_data = pd.read_csv(z_score_data_path)
```

- **`pd.read_csv()`**:
  - Fungsi dari library `pandas` untuk membaca file CSV dan mengubahnya menjadi DataFrame, format yang mudah digunakan untuk analisis data.

1. **`growth_data`**:
   - Membaca dataset `gizi_anak_indonesia_balanced_1000.csv` ke dalam DataFrame `growth_data`.
   - Dataset ini berisi fitur-fitur seperti usia, tinggi badan, berat badan, dan status gizi anak.
   - Data ini digunakan sebagai input utama untuk membuat model prediksi status gizi anak.

2. **`food_data`**:
   - Membaca dataset `FOOD-DATA-GROUP5-CLEANED.csv` ke dalam DataFrame `food_data`.
   - Dataset ini berisi informasi makanan seperti nilai kalori, kandungan protein, lemak, vitamin, dan mineral.
   - Data ini biasanya digunakan untuk memberikan rekomendasi makanan berdasarkan status gizi anak.

3. **`z_score_data`**:
   - Membaca dataset `z_score_data_who.csv` ke dalam DataFrame `z_score_data`.
   - Dataset ini memuat data z-score referensi WHO, seperti BMI rata-rata (`Mean BMI`) dan standar deviasi (`SD BMI`) berdasarkan usia dan jenis kelamin anak.
   - Digunakan untuk menganalisis apakah BMI anak berada dalam kisaran yang sehat berdasarkan data standar WHO.

**Catatan**:
- Baris `z_score_data_path` masih di-comment sehingga dataset ini tidak akan dibaca sampai comment dihapus. Pastikan untuk menghapus `#` jika dataset tersebut dibutuhkan.

---

### **Cara Kerja Keseluruhan**
1. Path dataset didefinisikan terlebih dahulu, sehingga kode tahu lokasi file yang akan dibaca.
2. `pandas.read_csv()` digunakan untuk membaca file dari lokasi tersebut.
3. Data dari file CSV diubah menjadi DataFrame, yang memungkinkan manipulasi data seperti perhitungan, normalisasi, atau penghapusan nilai yang tidak diperlukan.

---

### **Kesalahan Umum**
1. **File Tidak Ditemukan**:
   - Pastikan file CSV ada di path yang telah disebutkan di Google Drive.
2. **Komentar pada `z_score_data_path`**:
   - Baris ini masih di-comment, sehingga jika ingin menggunakan `z_score_data`, comment harus dihapus.
3. **Format CSV Tidak Valid**:
   - Pastikan dataset berbentuk CSV dan memiliki format yang sesuai (header, delimiter, dll.).

---


In [ ]:
growth_data.head()

,Usia (Tahun),Jenis Kelamin,Tinggi Badan (cm),Berat Badan (kg),BMI,Status Gizi,Kalori,Protein,Lemak,Karbohidrat,Vitamin C,Zat Besi,Kalsium,Riwayat Penyakit
0,11,Perempuan,144.7,25.2,15.13,Gizi Lebih,2395,70,17,114,71,17.82,1026,Diare
1,5,Laki-laki,133.4,34.4,17.54,Gizi Kurang,2416,10,27,139,143,16.37,1360,Stunting
2,17,Perempuan,140.0,27.6,25.68,Gizi Kurang,2838,46,12,151,113,12.38,649,Stunting
3,17,Perempuan,115.3,40.2,26.75,Gizi Lebih,1591,88,6,230,54,9.43,907,Diare
4,12,Laki-laki,115.0,37.1,17.83,Gizi Kurang,2539,67,58,116,116,5.56,1383,Infeksi Saluran Pernafasan


#### **Fungsi Utama**
Kode ini digunakan untuk menampilkan **5 baris pertama** dari dataset `growth_data`.

---

#### **Penjelasan Kode**
1. **`growth_data`**:
   - Adalah sebuah **DataFrame** yang sebelumnya dibaca dari file CSV menggunakan fungsi `pd.read_csv()`.
   - Dataset ini memuat informasi terkait status gizi anak, seperti:
     - Usia anak.
     - Tinggi badan.
     - Berat badan.
     - Status gizi (contohnya: "Gizi Kurang", "Gizi Baik", atau "Gizi Lebih").

2. **`.head()`**:
   - Adalah metode bawaan `pandas` yang digunakan untuk menampilkan beberapa baris pertama dari DataFrame.
   - Secara default, metode ini menampilkan **5 baris pertama**. Namun, Anda bisa menentukan jumlah baris yang ingin ditampilkan dengan menambahkan parameter, seperti `growth_data.head(10)` untuk 10 baris.

---

#### **Tujuan Penggunaan**
- **Memverifikasi Data**:
  - Memastikan dataset telah berhasil dimuat dengan benar ke dalam DataFrame.
  - Melihat struktur data, termasuk kolom-kolom yang tersedia dan beberapa nilai awal di dalamnya.

- **Mengidentifikasi Masalah**:
  - Melihat apakah ada nilai yang hilang (*missing values*), format data yang salah, atau inkonsistensi pada data.

In [ ]:
food_data.head()

,food,caloric_value,fat,saturated_fats,monounsaturated_fats,polyunsaturated_fats,carbohydrates,sugars,protein,dietary_fiber,...,calcium,copper,iron,magnesium,manganese,phosphorus,potassium,selenium,zinc,nutrition_density
0,margarine with yoghurt,88,9.8,1.9,5.6,2.0,0.073,0.0,0.058,0.0,...,2.8,0.001,0.027,0.3,0.0,2.2,3.5,0.000,0.008,12.971
1,sunflower seed butter,99,8.8,0.7,6.2,1.6,3.700,1.7,2.800,0.9,...,10.2,0.300,0.700,49.8,0.3,106.6,92.2,0.075,0.800,27.500
2,hazelnut oil,120,13.6,1.0,10.6,1.4,0.000,0.0,0.000,0.0,...,0.0,0.000,0.000,0.0,0.0,0.0,0.0,0.000,0.000,13.600
3,menhaden fish oil,1966,218.0,66.3,58.2,74.5,0.000,0.0,0.000,0.0,...,0.0,0.000,0.000,0.0,0.0,0.0,0.0,0.000,0.000,218.000
4,cod liver fish oil,123,13.6,3.1,6.4,3.1,0.000,0.0,0.000,0.0,...,0.0,0.000,0.000,0.0,0.0,0.0,0.0,0.000,0.000,17.700


#### **Fungsi Utama**
Kode ini digunakan untuk menampilkan **5 baris pertama** dari dataset `food_data`.

---

#### **Penjelasan Kode**
1. **`food_data`**:
   - Adalah sebuah **DataFrame** yang sebelumnya dibaca dari file CSV menggunakan fungsi `pd.read_csv()`.
   - Dataset ini memuat informasi terkait kandungan dari suatu makanan atau minuman.

2. **`.head()`**:
   - Adalah metode bawaan `pandas` yang digunakan untuk menampilkan beberapa baris pertama dari DataFrame.
   - Secara default, metode ini menampilkan **5 baris pertama**. Namun, Anda bisa menentukan jumlah baris yang ingin ditampilkan dengan menambahkan parameter, seperti `food_data.head(10)` untuk 10 baris.

---

#### **Tujuan Penggunaan**
- **Memverifikasi Data**:
  - Memastikan dataset telah berhasil dimuat dengan benar ke dalam DataFrame.
  - Melihat struktur data, termasuk kolom-kolom yang tersedia dan beberapa nilai awal di dalamnya.

- **Mengidentifikasi Masalah**:
  - Melihat apakah ada nilai yang hilang (*missing values*), format data yang salah, atau inkonsistensi pada data.

In [ ]:
print(growth_data['Status Gizi'].value_counts())

Status Gizi
Gizi Baik      334
Gizi Lebih     333
Gizi Kurang    333
Name: count, dtype: int64


---

#### **Fungsi Utama**
Kode ini digunakan untuk menghitung dan menampilkan jumlah kemunculan (*frekuensi*) setiap kategori dalam kolom **`Status Gizi`** pada dataset `growth_data`.

---

#### **Penjelasan Kode**
1. **`growth_data`**:
   - Merupakan sebuah **DataFrame** yang memuat data terkait status gizi anak.
   - DataFrame ini sebelumnya dibaca dari file CSV menggunakan `pd.read_csv()`.

2. **`growth_data['Status Gizi']`**:
   - Menunjuk ke kolom **`Status Gizi`** di dalam DataFrame `growth_data`.
   - Kolom ini berisi kategori status gizi anak, misalnya:
     - "Gizi Kurang".
     - "Gizi Baik".
     - "Gizi Lebih".

3. **`.value_counts()`**:
   - Fungsi bawaan `pandas` yang menghitung jumlah kemunculan unik dari setiap nilai/kategori dalam kolom `Status Gizi`.
   - Output berupa daftar kategori dengan jumlah frekuensi kemunculannya, diurutkan dari yang paling banyak hingga yang paling sedikit.

4. **`print()`**:
   - Digunakan untuk mencetak hasil dari `.value_counts()` ke layar.

---

#### **Tujuan Penggunaan**
- **Analisis Data Awal**:
  - Untuk mengetahui distribusi data pada kolom **`Status Gizi`**.
  - Memastikan apakah dataset memiliki distribusi data yang seimbang atau tidak, misalnya apakah jumlah data untuk kategori "Gizi Kurang", "Gizi Baik", dan "Gizi Lebih" cukup merata.

- **Identifikasi Masalah Dataset**:
  - Menemukan ketidakseimbangan (*class imbalance*), di mana jumlah data dalam satu kategori jauh lebih besar/kecil dibandingkan kategori lain.
  - Ketidakseimbangan ini penting diperhatikan karena dapat memengaruhi performa model prediksi.

---

In [ ]:
# get data info
growth_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Usia (Tahun)       1000 non-null   int64  
 1   Jenis Kelamin      1000 non-null   object 
 2   Tinggi Badan (cm)  1000 non-null   float64
 3   Berat Badan (kg)   1000 non-null   float64
 4   BMI                1000 non-null   float64
 5   Status Gizi        1000 non-null   object 
 6   Kalori             1000 non-null   int64  
 7   Protein            1000 non-null   int64  
 8   Lemak              1000 non-null   int64  
 9   Karbohidrat        1000 non-null   int64  
 10  Vitamin C          1000 non-null   int64  
 11  Zat Besi           1000 non-null   float64
 12  Kalsium            1000 non-null   int64  
 13  Riwayat Penyakit   1000 non-null   object 
dtypes: float64(4), int64(7), object(3)
memory usage: 109.5+ KB


---

#### **Fungsi Utama**
Kode ini digunakan untuk menampilkan informasi ringkas tentang struktur dataset **`growth_data`**, termasuk tipe data, jumlah baris, kolom, nilai yang tidak kosong (*non-null*), dan ukuran memori yang digunakan.

---

#### **Penjelasan Kode**
1. **`growth_data`**:
   - **`growth_data`** adalah sebuah objek **DataFrame** yang berisi dataset terkait data pertumbuhan dan status gizi anak.
   - Objek ini dibuat sebelumnya menggunakan `pd.read_csv()` untuk membaca dataset dari file CSV.

2. **`.info()`**:
   - Fungsi bawaan `pandas` yang memberikan gambaran umum tentang DataFrame.
   - Informasi yang ditampilkan mencakup:
     - **Nama kolom**: Menunjukkan nama setiap kolom dalam dataset.
     - **Tipe data**: Menunjukkan tipe data setiap kolom, seperti integer (`int64`), float (`float64`), atau string (`object`).
     - **Jumlah nilai non-null**: Menampilkan jumlah nilai yang tidak kosong di setiap kolom.
     - **Jumlah total baris**: Menunjukkan jumlah baris dalam dataset.
     - **Penggunaan memori**: Menunjukkan jumlah memori yang digunakan oleh DataFrame.

3. **Hasil Fungsi**:
   - Hasil dari fungsi `.info()` dicetak langsung ke layar tanpa perlu menggunakan `print()`.

---

#### **Tujuan Penggunaan**
1. **Memahami Struktur Dataset**:
   - Untuk memahami jumlah baris dan kolom dalam dataset, serta mengetahui tipe data setiap kolom.

2. **Pengecekan Kualitas Data**:
   - Untuk memastikan tidak ada nilai yang hilang (*missing values*) pada dataset.
   - Jika terdapat nilai kosong di suatu kolom, jumlah nilai non-null akan lebih kecil dari total jumlah baris.

3. **Debugging Dataset**:
   - Untuk memeriksa apakah setiap kolom memiliki tipe data yang sesuai. Misalnya, kolom "Tinggi Badan" seharusnya memiliki tipe data numerik (bukan string).

---

In [ ]:
# get data info
food_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 722 entries, 0 to 721
Data columns (total 35 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   food                  722 non-null    object 
 1   caloric_value         722 non-null    int64  
 2   fat                   722 non-null    float64
 3   saturated_fats        722 non-null    float64
 4   monounsaturated_fats  722 non-null    float64
 5   polyunsaturated_fats  722 non-null    float64
 6   carbohydrates         722 non-null    float64
 7   sugars                722 non-null    float64
 8   protein               722 non-null    float64
 9   dietary_fiber         722 non-null    float64
 10  cholesterol           722 non-null    float64
 11  sodium                722 non-null    float64
 12  water                 722 non-null    float64
 13  vitamin_a             722 non-null    float64
 14  vitamin_b1            722 non-null    float64
 15  vitamin_b11           7

---

#### **Fungsi Utama**
Kode ini digunakan untuk menampilkan informasi ringkas tentang struktur dataset **`food_data`**, termasuk tipe data, jumlah baris, kolom, nilai yang tidak kosong (*non-null*), dan ukuran memori yang digunakan.

---

#### **Penjelasan Kode**
1. **`food_data`**:
   - **`food_data`** adalah sebuah objek **DataFrame** yang berisi dataset terkait data kandungan pada makanan atau minuman.
   - Objek ini dibuat sebelumnya menggunakan `pd.read_csv()` untuk membaca dataset dari file CSV.

2. **`.info()`**:
   - Fungsi bawaan `pandas` yang memberikan gambaran umum tentang DataFrame.
   - Informasi yang ditampilkan mencakup:
     - **Nama kolom**: Menunjukkan nama setiap kolom dalam dataset.
     - **Tipe data**: Menunjukkan tipe data setiap kolom, seperti integer (`int64`), float (`float64`), atau string (`object`).
     - **Jumlah nilai non-null**: Menampilkan jumlah nilai yang tidak kosong di setiap kolom.
     - **Jumlah total baris**: Menunjukkan jumlah baris dalam dataset.
     - **Penggunaan memori**: Menunjukkan jumlah memori yang digunakan oleh DataFrame.

3. **Hasil Fungsi**:
   - Hasil dari fungsi `.info()` dicetak langsung ke layar tanpa perlu menggunakan `print()`.

---


In [ ]:
growth_data.drop_duplicates(inplace=True)

---

#### **Fungsi Utama**
Kode ini digunakan untuk **menghapus baris duplikat** dari dataset **`growth_data`**, sehingga hanya menyisakan baris-baris yang unik.

---

#### **Penjelasan Kode**
1. **`growth_data`**:
   - Objek ini adalah **DataFrame** yang sebelumnya diisi dengan data dari dataset menggunakan fungsi `pd.read_csv()`.

2. **`drop_duplicates()`**:
   - Fungsi bawaan dari `pandas` untuk menghapus baris yang memiliki nilai duplikat di seluruh kolom dataset.
   - Baris dianggap duplikat jika semua nilai dalam baris tersebut identik dengan baris lain dalam dataset.

3. **`inplace=True`**:
   - Parameter ini memastikan bahwa perubahan dilakukan langsung pada **DataFrame asli** (`growth_data`), tanpa perlu membuat salinan baru.
   - Jika **`inplace=False`** (default), maka fungsi akan mengembalikan DataFrame baru tanpa duplikat, sementara DataFrame asli tetap tidak berubah.

---

#### **Tujuan Penggunaan**
1. **Meningkatkan Akurasi Analisis**:
   - Duplikat data dapat menyebabkan bias atau kesalahan dalam analisis, seperti perhitungan rata-rata, distribusi data, atau pelatihan model machine learning.
   - Dengan menghapus duplikat, dataset menjadi lebih bersih dan analisis menjadi lebih akurat.

2. **Mengurangi Ukuran Dataset**:
   - Dataset dengan duplikat data membutuhkan lebih banyak memori.
   - Menghapus duplikat dapat mengurangi ukuran dataset dan meningkatkan efisiensi pemrosesan.

3. **Menghindari Redundansi**:
   - Dalam analisis data atau pelatihan model, keberadaan duplikat bisa menyebabkan hasil yang tidak konsisten atau berulang.

---


In [ ]:
food_data.drop_duplicates(inplace=True)

---

#### **Fungsi Utama**
Kode ini digunakan untuk **menghapus baris duplikat** dari dataset **`food_data`**, sehingga hanya menyisakan baris-baris yang unik.

---

#### **Penjelasan Kode**
1. **`food_data`**:
   - Objek ini adalah **DataFrame** yang sebelumnya diisi dengan data dari dataset menggunakan fungsi `pd.read_csv()`.

2. **`drop_duplicates()`**:
   - Fungsi bawaan dari `pandas` untuk menghapus baris yang memiliki nilai duplikat di seluruh kolom dataset.
   - Baris dianggap duplikat jika semua nilai dalam baris tersebut identik dengan baris lain dalam dataset.

3. **`inplace=True`**:
   - Parameter ini memastikan bahwa perubahan dilakukan langsung pada **DataFrame asli** (`food_data`), tanpa perlu membuat salinan baru.
   - Jika **`inplace=False`** (default), maka fungsi akan mengembalikan DataFrame baru tanpa duplikat, sementara DataFrame asli tetap tidak berubah.

---

#### **Tujuan Penggunaan**
1. **Meningkatkan Akurasi Analisis**:
   - Duplikat data dapat menyebabkan bias atau kesalahan dalam analisis, seperti perhitungan rata-rata, distribusi data, atau pelatihan model machine learning.
   - Dengan menghapus duplikat, dataset menjadi lebih bersih dan analisis menjadi lebih akurat.

2. **Mengurangi Ukuran Dataset**:
   - Dataset dengan duplikat data membutuhkan lebih banyak memori.
   - Menghapus duplikat dapat mengurangi ukuran dataset dan meningkatkan efisiensi pemrosesan.

3. **Menghindari Redundansi**:
   - Dalam analisis data atau pelatihan model, keberadaan duplikat bisa menyebabkan hasil yang tidak konsisten atau berulang.

---


In [ ]:
# **Fitur yang Digunakan**
food_features = [
    'caloric_value', 'fat', 'saturated_fats', 'monounsaturated_fats',
    'polyunsaturated_fats', 'carbohydrates', 'sugars', 'protein',
    'dietary_fiber', 'cholesterol', 'sodium', 'water', 'vitamin_a',
    'vitamin_b1', 'vitamin_b11', 'vitamin_b12', 'vitamin_b2', 'vitamin_b3',
    'vitamin_b5', 'vitamin_b6', 'vitamin_c', 'vitamin_d', 'vitamin_e',
    'vitamin_k', 'calcium', 'copper', 'iron', 'magnesium', 'manganese',
    'phosphorus', 'potassium', 'selenium', 'zinc', 'nutrition_density'
]

# **Normalisasi Data**
scaler_food = MinMaxScaler()
normalized_food_features = scaler_food.fit_transform(food_data[food_features])

# **Model Rekomendasi**
# Input preferensi pengguna dan data makanan
user_input = Input(shape=(len(food_features),), name="user_input")
food_input = Input(shape=(len(food_features),), name="food_input")

# Arsitektur Model dengan Dropout dan LeakyReLU
concatenated = Concatenate()([user_input, food_input])

dense1 = Dense(512, activation='relu')(concatenated)
dense1 = BatchNormalization()(dense1)
dense1 = Dropout(0.3)(dense1)

dense2 = Dense(256, activation='relu')(dense1)
dense2 = BatchNormalization()(dense2)
dense2 = Dropout(0.2)(dense2)

dense3 = Dense(128, activation='relu')(dense2)
dense3 = BatchNormalization()(dense3)
dense3 = Dropout(0.2)(dense3)

output = Dense(1, activation='relu')(dense3)

recommendation_model = Model(inputs=[user_input, food_input], outputs=output)
recommendation_model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='mean_squared_error',
    metrics=['mae', 'accuracy']
)

# **Generate Training Data**
user_preferences = np.random.uniform(0, 1, size=(len(food_data), len(food_features)))
ratings = np.random.uniform(1, 5, size=(len(food_data), 1))

# Split data menjadi training dan validation
X_train_user, X_val_user, X_train_food, X_val_food, y_train_rec, y_val_rec = train_test_split(
    user_preferences, normalized_food_features, ratings, test_size=0.2, random_state=42
)

# Train Model
history_rec = recommendation_model.fit(
    [X_train_user, X_train_food], y_train_rec,
    validation_data=([X_val_user, X_val_food], y_val_rec),
    epochs=100,
    batch_size=32,
    verbose=1
)

# Evaluasi model rekomendasi makanan
y_pred_rec = recommendation_model.predict([X_val_user, X_val_food]).flatten()

# Hitung Mean Squared Error (MSE) dan Mean Absolute Error (MAE)
mse_rec = mean_squared_error(y_val_rec, y_pred_rec)
mae_rec = mean_absolute_error(y_val_rec, y_pred_rec)

print(f"MSE Model Rekomendasi Makanan: {mse_rec:.4f}")
print(f"MAE Model Rekomendasi Makanan: {mae_rec:.4f}")

# Konversi MAE ke persen
mean_rating = np.mean(y_val_rec)
percentage_error = (mae_rec / mean_rating) * 100
print(f"Kesalahan Rata-rata Model Rekomendasi Makanan: {percentage_error:.2f}%")

# Visualisasi Akurasi
plt.plot(history_rec.history['accuracy'], label='Training Accuracy')
plt.plot(history_rec.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Visualisasi Loss
plt.plot(history_rec.history['loss'], label='Training Loss')
plt.plot(history_rec.history['val_loss'], label='Validation Loss')
plt.title('Loss During Training')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()
